# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df=pd.read_csv('output_data/weather.txt')
df


,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,41.49,90,100,3.49,NZ,1626810709
1,1,Brae,60.3964,-1.3530,53.51,81,17,5.44,GB,1626810886
2,2,Yamada,39.4667,141.9500,73.17,91,42,3.00,JP,1626810886
3,3,Coquimbo,-29.9533,-71.3436,55.13,76,20,11.50,CL,1626810887
4,4,Shingū,33.7333,135.9833,73.90,92,38,6.60,JP,1626810887
...,...,...,...,...,...,...,...,...,...,...
595,595,Riyadh,24.6877,46.7219,98.74,18,100,5.50,SA,1626810976
596,596,Zhumadian,32.9794,114.0294,74.68,96,100,1.63,CN,1626811243
597,597,Stepnogorsk,52.3544,71.8869,62.76,72,5,7.47,KZ,1626811243
598,598,Lagoa,39.0500,-27.9833,68.52,88,2,22.68,PT,1626811245


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
locations = df[["Lat", "Lon"]]

# Fill NaN values and convert to float
weight = df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
df=df.dropna(how='any')
df

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,41.49,90,100,3.49,NZ,1626810709
1,1,Brae,60.3964,-1.3530,53.51,81,17,5.44,GB,1626810886
2,2,Yamada,39.4667,141.9500,73.17,91,42,3.00,JP,1626810886
3,3,Coquimbo,-29.9533,-71.3436,55.13,76,20,11.50,CL,1626810887
4,4,Shingū,33.7333,135.9833,73.90,92,38,6.60,JP,1626810887
...,...,...,...,...,...,...,...,...,...,...
595,595,Riyadh,24.6877,46.7219,98.74,18,100,5.50,SA,1626810976
596,596,Zhumadian,32.9794,114.0294,74.68,96,100,1.63,CN,1626811243
597,597,Stepnogorsk,52.3544,71.8869,62.76,72,5,7.47,KZ,1626811243
598,598,Lagoa,39.0500,-27.9833,68.52,88,2,22.68,PT,1626811245


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=df
hotel_df['Hotel Name']=''
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lon=row['Lon']
    params = {
    "location": f"{lat},{lon}",  # philadelphia coords
    "radius": 50000,
    "types": "hotel",
    "key": g_key,
    }
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address["results"][0]["name"]
        name=hotel_df.loc[index,'Hotel Name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lat', 'Lon']]

# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(marker_locations,info_box_content=f"Hotel Name: {name}")
fig.add_layer(markers)
fig

<ipython-input-6-24481f57bc62>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=''
C:\Users\frank\anaconda\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Invercargill
Lerwick
Morioka
Coquimbo
Shingu
Guerrero Negro
Banda Aceh
Ushuaia
Winton
Corpus Christi
Barnstaple
Chokurdakh
Port Louis
Invercargill
Carnarvon
Saint Petersburg
Cape Town
Tórshavn
Kapaʻa
Albany
Datong
Praia
Port Louis
Uturoa
Gävle
Turkmenabat
Provideniya
Mwinilunga
Qaqortoq
Bataraza
Chiclayo
Dunedin
Curanilahue
Hobart
Esperance
Puerto Ayora
Cape Town
Bunbury
Porto
Namatanai
Coos Bay
Saskylakh
Dikson
Potsdam
Port Macquarie
Nuku'alofa
Gqeberha
Saint-Paul
Kaitaia
Ilulissat
Tuktoyaktuk
Batsfjord
Singkawang
Isangel
Hobart
Nova Viçosa
Angra do Heroísmo
Horta
Trujillo
Verkhoyansk
Thompson
Jamestown
Niaqornaarsuk
Norilsk
Cayenne
Santa Vitoria do Palmar
Trelew
Kamisu
Denpasar
Oulu
George Town
East London
Baruun-Urt
Betul
Cincinnati
Bandarbeyla
Avarua District
Hilo
Tres Arroyos
Boa Vista
Ukiangang
Sosnovo
Punta Arenas
Akhtubinsk
Port-Gentil
Broken Hill
Vestmannaeyjar
Tabou
Copiapó
Mapiripán
Derzhavinsk
Rikitea
Bredasdorp
Leningradsky
Te Anau
Bengkulu
Ponta Grossa
Dunedin
Viedma
Mar 

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "['Lng'] not in index"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
